# Secure Split-FL Testing on Google Colab

This notebook sets up and runs the Secure Split-FL tests using GPU acceleration on Google Colab.

## Setup Instructions
1. Upload this notebook to Google Colab
2. Make sure you have selected GPU runtime (Runtime -> Change runtime type -> GPU)
3. Run each cell in order

In [ ]:
# Check if GPU is available
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU device: {torch.cuda.get_device_name(0)}")

In [ ]:
# Clone the repository
!git clone https://github.com/YOUR_USERNAME/Comp430_Project.git
%cd Comp430_Project

In [ ]:
# Install required packages
!pip install -r requirements.txt pytest

In [ ]:
# Create necessary directories
!mkdir -p experiments/out
!mkdir -p experiments/tests

In [ ]:
%%writefile experiments/tests/utils.py
import json
import os
import pathlib
import shutil
import subprocess
import uuid
import yaml
import datetime

ROOT = pathlib.Path(__file__).resolve().parents[1]
OUT  = ROOT / "out"

def run_experiment(cfg_path: str):
    """Launch one SFL run, return metrics dict."""
    run_id = f"{pathlib.Path(cfg_path).stem}__{uuid.uuid4().hex[:6]}"
    run_dir = OUT / run_id
    run_dir.mkdir(parents=True, exist_ok=True)

    # 1. spawn training as subprocess
    cmd = [
        "python", "experiments/train_secure_sfl.py",
        "--config", cfg_path,
        "--run_id", run_id                  
    ]
    result = subprocess.run(cmd, capture_output=True, text=True, check=True)

    # 2. collect metrics saved by the runner
    metrics_file = run_dir / "metrics.json"
    with open(metrics_file) as f:
        metrics = json.load(f)

    # 3. additionally log stdout / stderr
    (run_dir / "stdout.txt").write_text(result.stdout)
    (run_dir / "stderr.txt").write_text(result.stderr)

    return metrics

In [ ]:
%%writefile experiments/tests/test_pipeline.py
from .utils import run_experiment

def test_forward_backward():
    """Tests that the pipeline works by running a basic experiment."""
    m = run_experiment("configs/default.yaml")
    assert 0.0 < m["final_test_acc"] < 1.0,  "accuracy not logged"
    assert m["rounds"] >= 1,                 "training never started"

In [ ]:
%%writefile experiments/tests/test_dp_budget.py
from .utils import run_experiment

MAX_EPS = 8.0
def test_fixed_dp():
    """Tests that the fixed DP budget does not exceed the maximum epsilon."""
    m = run_experiment("configs/fixed_dp.yaml")
    assert m["epsilon"] <= MAX_EPS, "ε budget exceeded"

def test_adaptive_sigma():
    """Tests that the adaptive noise mechanism reduces noise over time."""
    m = run_experiment("configs/adaptive_dp.yaml")
    assert m["sigma"] < m["sigma_init"], "σ did not decay"

In [ ]:
%%writefile experiments/tests/test_fedavg_equiv.py
import torch
from .utils import run_experiment

THRESH = 1e-3
def test_single_client_equivalence():
    """Tests that 1-client SFL is equivalent to centralized training."""
    m_split = run_experiment("configs/one_client.yaml")
    m_central = run_experiment("configs/central.yaml")
    diff = abs(m_split["final_test_acc"] - m_central["final_test_acc"])
    assert diff < THRESH, f"Split = {m_split}, Central = {m_central}"

In [ ]:
%%writefile experiments/tests/test_utility.py
import yaml
from .utils import run_experiment

def test_minimum_accuracy():
    """Tests that the model achieves the minimum accuracy specified in the config file."""
    config_path = "configs/dnn_default.yaml"
    with open(config_path, 'r') as f:
        config = yaml.safe_load(f)
    
    min_acc = config.get('min_acc', 0.7)
    
    m = run_experiment(config_path)
    assert m["final_test_acc"] >= min_acc, f"Model accuracy {m['final_test_acc']} below minimum threshold {min_acc}"

In [ ]:
%%writefile experiments/tests/test_known_config.py
from .utils import run_experiment

def test_known_working_config():
    """Tests the known working configuration (cnn_adaptive_dp.yaml)."""
    m = run_experiment("configs/cnn_adaptive_dp.yaml")
    assert m["final_test_acc"] > 0.70, "Expected accuracy of at least 70% with this config"
    assert m["sigma_init"] == 0.0 or m["sigma"] == 0.0, "DP noise should be disabled in this test"

In [ ]:
%%writefile experiments/tests/__init__.py
# Empty init file

## Run Individual Tests
You can run specific test files to check different aspects of the system:

In [ ]:
# Run basic pipeline test
!pytest -v experiments/tests/test_pipeline.py

In [ ]:
# Run DP budget tests
!pytest -v experiments/tests/test_dp_budget.py

In [ ]:
# Run federated averaging equivalence test
!pytest -v experiments/tests/test_fedavg_equiv.py

In [ ]:
# Run utility test
!pytest -v experiments/tests/test_utility.py

In [ ]:
# Run known configuration test
!pytest -v experiments/tests/test_known_config.py

## Run All Tests
Run all tests at once to verify the complete system:

In [ ]:
# Run all tests
!pytest -v experiments/tests/